# 03 - Model Training & Evaluation
## Real Estate Price Prediction

**Author:** Nicolas  
**Date:** 2025-01-09  
**Objective:** Entraîner, comparer et optimiser plusieurs modèles ML pour la prédiction de prix immobiliers

---

### Table of Contents
1. [Data Loading](#1-data-loading)
2. [Baseline Model](#2-baseline-model)
3. [Linear Models](#3-linear-models)
4. [Tree-Based Models](#4-tree-based-models)
5. [Ensemble Methods](#5-ensemble-methods)
6. [Model Comparison](#6-model-comparison)
7. [Hyperparameter Tuning](#7-hyperparameter-tuning)
8. [Final Model Evaluation](#8-final-model-evaluation)
9. [Feature Importance](#9-feature-importance)
10. [Model Interpretation (SHAP)](#10-model-interpretation)
11. [Model Persistence](#11-model-persistence)

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import joblib
import json
from time import time
import warnings

# Scikit-learn
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

# Advanced ML
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

# Model interpretation
import shap

warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)

%matplotlib inline

# Seed pour reproductibilité
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 1. Data Loading

In [ ]:
# Charger les données préparées
print("=" * 80)
print("CHARGEMENT DES DONNÉES")
print("=" * 80)

with open('../data/processed_data.pkl', 'rb') as f:
    data = pickle.load(f)

X_train = data['X_train']
X_test = data['X_test']
y_train = data['y_train']
y_test = data['y_test']
y_train_log = data['y_train_log']
y_test_log = data['y_test_log']
feature_names = data['feature_names']

print(f"\n✅ Données chargées:")
print(f"  X_train: {X_train.shape}")
print(f"  X_test:  {X_test.shape}")
print(f"  Features: {len(feature_names)}")
print(f"\n  Target statistics (train):")
print(f"    Mean:   {y_train.mean():,.2f} €")
print(f"    Median: {y_train.median():,.2f} €")
print(f"    Std:    {y_train.std():,.2f} €")

In [ ]:
# Fonction pour évaluer les modèles
def evaluate_model(y_true, y_pred, model_name="Model"):
    """
    Calcule et affiche les métriques de régression.
    """
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n{model_name} - Métriques:")
    print(f"  RMSE:  {rmse:,.2f} €")
    print(f"  MAE:   {mae:,.2f} €")
    print(f"  MAPE:  {mape:.2f}%")
    print(f"  R²:    {r2:.4f}")
    
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'R2': r2}

# Dictionnaire pour stocker les résultats
results = {}

## 2. Baseline Model

In [ ]:
print("=" * 80)
print("BASELINE MODEL - Prédiction par la moyenne")
print("=" * 80)

# Baseline: prédire toujours la moyenne du train set
y_pred_baseline = np.full(len(y_test), y_train.mean())
results['Baseline (Mean)'] = evaluate_model(y_test, y_pred_baseline, "Baseline (Mean)")

print("\n💡 Le baseline nous donne un point de référence.")
print("   Tout modèle doit faire significativement mieux que cela!")

## 3. Linear Models

In [ ]:
print("\n" + "=" * 80)
print("MODÈLES LINÉAIRES")
print("=" * 80)

# 3.1 Linear Regression
print("\n[1/4] Linear Regression...")
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
results['Linear Regression'] = evaluate_model(y_test, y_pred_lr, "Linear Regression")

# 3.2 Ridge Regression (L2 regularization)
print("\n[2/4] Ridge Regression...")
ridge = Ridge(alpha=10.0, random_state=RANDOM_STATE)
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)
results['Ridge'] = evaluate_model(y_test, y_pred_ridge, "Ridge")

# 3.3 Lasso Regression (L1 regularization)
print("\n[3/4] Lasso Regression...")
lasso = Lasso(alpha=10.0, random_state=RANDOM_STATE, max_iter=10000)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)
results['Lasso'] = evaluate_model(y_test, y_pred_lasso, "Lasso")

# 3.4 ElasticNet (L1 + L2)
print("\n[4/4] ElasticNet...")
elastic = ElasticNet(alpha=10.0, l1_ratio=0.5, random_state=RANDOM_STATE, max_iter=10000)
elastic.fit(X_train, y_train)
y_pred_elastic = elastic.predict(X_test)
results['ElasticNet'] = evaluate_model(y_test, y_pred_elastic, "ElasticNet")

print("\n✅ Modèles linéaires entraînés")

## 4. Tree-Based Models

In [ ]:
print("\n" + "=" * 80)
print("MODÈLES À BASE D'ARBRES")
print("=" * 80)

# 4.1 Decision Tree
print("\n[1/2] Decision Tree...")
dt = DecisionTreeRegressor(max_depth=10, min_samples_split=20, random_state=RANDOM_STATE)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
results['Decision Tree'] = evaluate_model(y_test, y_pred_dt, "Decision Tree")

# 4.2 Random Forest
print("\n[2/2] Random Forest...")
rf = RandomForestRegressor(n_estimators=100, max_depth=15, min_samples_split=10, 
                           random_state=RANDOM_STATE, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
results['Random Forest'] = evaluate_model(y_test, y_pred_rf, "Random Forest")

print("\n✅ Modèles à base d'arbres entraînés")

## 5. Ensemble Methods (Boosting)

In [ ]:
print("\n" + "=" * 80)
print("MÉTHODES D'ENSEMBLE (BOOSTING)")
print("=" * 80)

# 5.1 Gradient Boosting
print("\n[1/4] Gradient Boosting...")
start = time()
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5,
                               random_state=RANDOM_STATE)
gb.fit(X_train, y_train)
y_pred_gb = gb.predict(X_test)
results['Gradient Boosting'] = evaluate_model(y_test, y_pred_gb, "Gradient Boosting")
print(f"  Training time: {time() - start:.2f}s")

# 5.2 XGBoost
print("\n[2/4] XGBoost...")
start = time()
xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5,
                             random_state=RANDOM_STATE, n_jobs=-1)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
results['XGBoost'] = evaluate_model(y_test, y_pred_xgb, "XGBoost")
print(f"  Training time: {time() - start:.2f}s")

# 5.3 LightGBM
print("\n[3/4] LightGBM...")
start = time()
lgb_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, max_depth=5,
                              random_state=RANDOM_STATE, n_jobs=-1, verbose=-1)
lgb_model.fit(X_train, y_train)
y_pred_lgb = lgb_model.predict(X_test)
results['LightGBM'] = evaluate_model(y_test, y_pred_lgb, "LightGBM")
print(f"  Training time: {time() - start:.2f}s")

# 5.4 CatBoost
print("\n[4/4] CatBoost...")
start = time()
cat_model = CatBoostRegressor(iterations=100, learning_rate=0.1, depth=5,
                              random_state=RANDOM_STATE, verbose=0)
cat_model.fit(X_train, y_train)
y_pred_cat = cat_model.predict(X_test)
results['CatBoost'] = evaluate_model(y_test, y_pred_cat, "CatBoost")
print(f"  Training time: {time() - start:.2f}s")

print("\n✅ Modèles de boosting entraînés")

## 6. Model Comparison

In [ ]:
print("\n" + "=" * 80)
print("COMPARAISON DES MODÈLES")
print("=" * 80)

# Créer un DataFrame de résultats
results_df = pd.DataFrame(results).T
results_df = results_df.sort_values('R2', ascending=False)

print("\n📊 Classement des modèles (par R²):")
print(results_df.to_string())

# Sauvegarder les résultats
results_df.to_csv('../models/model_comparison.csv')
print("\n✅ Résultats sauvegardés: ../models/model_comparison.csv")

In [ ]:
# Visualisation des performances
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# R² Score
results_df['R2'].plot(kind='barh', ax=axes[0, 0], color='skyblue', edgecolor='black')
axes[0, 0].set_title('R² Score (plus élevé = meilleur)', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('R²')
axes[0, 0].axvline(0.9, color='green', linestyle='--', alpha=0.5, label='Excellent (>0.9)')
axes[0, 0].legend()

# RMSE
results_df['RMSE'].sort_values().plot(kind='barh', ax=axes[0, 1], color='coral', edgecolor='black')
axes[0, 1].set_title('RMSE (plus bas = meilleur)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('RMSE (€)')

# MAE
results_df['MAE'].sort_values().plot(kind='barh', ax=axes[1, 0], color='lightgreen', edgecolor='black')
axes[1, 0].set_title('MAE (plus bas = meilleur)', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('MAE (€)')

# MAPE
results_df['MAPE'].sort_values().plot(kind='barh', ax=axes[1, 1], color='gold', edgecolor='black')
axes[1, 1].set_title('MAPE (plus bas = meilleur)', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('MAPE (%)')
axes[1, 1].axvline(10, color='green', linestyle='--', alpha=0.5, label='Excellent (<10%)')
axes[1, 1].legend()

plt.tight_layout()
plt.savefig('../models/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Graphique sauvegardé: ../models/model_comparison.png")

In [ ]:
# Identifier le meilleur modèle
best_model_name = results_df.index[0]
best_r2 = results_df.iloc[0]['R2']
best_rmse = results_df.iloc[0]['RMSE']

print("\n" + "=" * 80)
print("🏆 MEILLEUR MODÈLE")
print("=" * 80)
print(f"\nModèle: {best_model_name}")
print(f"R²:     {best_r2:.4f}")
print(f"RMSE:   {best_rmse:,.2f} €")
print(f"\n💡 Ce modèle sera optimisé dans la section suivante")

## 7. Hyperparameter Tuning

In [ ]:
print("\n" + "=" * 80)
print("OPTIMISATION DES HYPERPARAMÈTRES (GridSearchCV)")
print("=" * 80)

# On va optimiser XGBoost (généralement le meilleur)
print("\nOptimisation de XGBoost...")

# Grille de paramètres
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

print(f"\nGrille de recherche: {len(param_grid['n_estimators']) * len(param_grid['max_depth']) * len(param_grid['learning_rate']) * len(param_grid['subsample']) * len(param_grid['colsample_bytree'])} combinaisons")
print("\n⚠️  Cela peut prendre plusieurs minutes...")

# GridSearchCV avec validation croisée
start = time()
grid_search = GridSearchCV(
    estimator=xgb.XGBRegressor(random_state=RANDOM_STATE, n_jobs=-1),
    param_grid=param_grid,
    scoring='r2',
    cv=5,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)
training_time = time() - start

print(f"\n✅ Optimisation terminée en {training_time:.2f}s ({training_time/60:.1f} minutes)")
print(f"\nMeilleurs paramètres:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

# Modèle optimisé
best_xgb = grid_search.best_estimator_
y_pred_best = best_xgb.predict(X_test)

print("\n" + "=" * 80)
print("COMPARAISON AVANT/APRÈS OPTIMISATION")
print("=" * 80)

print("\nXGBoost (défaut):")
for metric, value in results['XGBoost'].items():
    print(f"  {metric}: {value:,.4f}" if metric == 'R2' else f"  {metric}: {value:,.2f}")

results['XGBoost (Optimized)'] = evaluate_model(y_test, y_pred_best, "\nXGBoost (Optimized)")

# Improvement
r2_improvement = (results['XGBoost (Optimized)']['R2'] - results['XGBoost']['R2']) / results['XGBoost']['R2'] * 100
rmse_improvement = (results['XGBoost']['RMSE'] - results['XGBoost (Optimized)']['RMSE']) / results['XGBoost']['RMSE'] * 100

print(f"\n📈 Amélioration:")
print(f"  R² : +{r2_improvement:.2f}%")
print(f"  RMSE: -{rmse_improvement:.2f}%")

## 8. Final Model Evaluation

In [ ]:
print("\n" + "=" * 80)
print("ÉVALUATION FINALE DU MEILLEUR MODÈLE")
print("=" * 80)

# Prédictions vs réalité
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# 1. Scatter plot: Prédictions vs Réalité
axes[0].scatter(y_test, y_pred_best, alpha=0.5, s=30)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
             'r--', lw=2, label='Prédiction parfaite')
axes[0].set_xlabel('Prix Réel (€)', fontsize=11)
axes[0].set_ylabel('Prix Prédit (€)', fontsize=11)
axes[0].set_title('Prédictions vs Réalité', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 2. Distribution des erreurs
errors = y_test - y_pred_best
axes[1].hist(errors, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(0, color='red', linestyle='--', linewidth=2, label='Erreur nulle')
axes[1].set_xlabel('Erreur de prédiction (€)', fontsize=11)
axes[1].set_ylabel('Fréquence', fontsize=11)
axes[1].set_title(f'Distribution des Erreurs (Moyenne: {errors.mean():,.0f}€)', 
                  fontsize=12, fontweight='bold')
axes[1].legend()

# 3. Erreurs relatives (%)
relative_errors = np.abs((y_test - y_pred_best) / y_test) * 100
axes[2].hist(relative_errors, bins=50, edgecolor='black', alpha=0.7, color='coral')
axes[2].axvline(10, color='green', linestyle='--', linewidth=2, label='Erreur 10%')
axes[2].set_xlabel('Erreur relative (%)', fontsize=11)
axes[2].set_ylabel('Fréquence', fontsize=11)
axes[2].set_title(f'Distribution des Erreurs Relatives (Médiane: {np.median(relative_errors):.1f}%)', 
                  fontsize=12, fontweight='bold')
axes[2].legend()

plt.tight_layout()
plt.savefig('../models/final_model_evaluation.png', dpi=300, bbox_inches='tight')
plt.show()

# Statistiques sur les erreurs
print("\nStatistiques sur les erreurs:")
print(f"  Erreur moyenne:         {errors.mean():,.2f} €")
print(f"  Erreur médiane:         {errors.median():,.2f} €")
print(f"  Écart-type des erreurs: {errors.std():,.2f} €")
print(f"\n  Erreur relative moyenne:  {relative_errors.mean():.2f}%")
print(f"  Erreur relative médiane:  {np.median(relative_errors):.2f}%")
print(f"  % de prédictions à ±10%:  {(relative_errors <= 10).sum() / len(relative_errors) * 100:.1f}%")
print(f"  % de prédictions à ±20%:  {(relative_errors <= 20).sum() / len(relative_errors) * 100:.1f}%")

## 9. Feature Importance

In [ ]:
print("\n" + "=" * 80)
print("IMPORTANCE DES FEATURES")
print("=" * 80)

# Extraire l'importance des features
feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': best_xgb.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 20 features les plus importantes:")
print(feature_importance.head(20).to_string(index=False))

# Visualisation
fig, ax = plt.subplots(figsize=(10, 12))
feature_importance.head(20).plot(kind='barh', x='feature', y='importance', 
                                 ax=ax, legend=False, edgecolor='black', alpha=0.7)
ax.set_title('Top 20 Features - Importance', fontsize=14, fontweight='bold')
ax.set_xlabel('Importance', fontsize=11)
ax.set_ylabel('Feature', fontsize=11)
ax.invert_yaxis()
plt.tight_layout()
plt.savefig('../models/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

# Sauvegarder
feature_importance.to_csv('../models/feature_importance.csv', index=False)
print("\n✅ Importance sauvegardée: ../models/feature_importance.csv")

## 10. Model Interpretation (SHAP)

In [ ]:
print("\n" + "=" * 80)
print("INTERPRÉTATION DU MODÈLE AVEC SHAP")
print("=" * 80)

print("\n⚠️  Calcul des SHAP values (peut prendre 1-2 minutes)...")

# Créer l'explainer SHAP
explainer = shap.TreeExplainer(best_xgb)

# Calculer SHAP values (sur un échantillon pour la rapidité)
sample_size = min(500, len(X_test))
X_test_sample = X_test.sample(n=sample_size, random_state=RANDOM_STATE)
shap_values = explainer.shap_values(X_test_sample)

print("✅ SHAP values calculées")

# 1. Summary plot (importance globale)
print("\nGénération du SHAP Summary Plot...")
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test_sample, show=False, max_display=20)
plt.title('SHAP Summary Plot - Impact des Features', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig('../models/shap_summary_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n💡 Interprétation:")
print("  - Chaque point représente une prédiction")
print("  - Couleur rouge = valeur élevée de la feature")
print("  - Couleur bleue = valeur faible de la feature")
print("  - Position droite = impact positif sur le prix")
print("  - Position gauche = impact négatif sur le prix")

In [ ]:
# 2. Force plot pour une prédiction individuelle
print("\n" + "=" * 80)
print("SHAP FORCE PLOT - Explication d'une prédiction")
print("=" * 80)

# Choisir un exemple
idx = 0
example = X_test_sample.iloc[idx]
example_shap = shap_values[idx]

print(f"\nExemple #{idx}:")
print(f"  Valeur réelle: {y_test.iloc[idx]:,.2f} €")
print(f"  Prédiction:    {best_xgb.predict(example.to_frame().T)[0]:,.2f} €")

# Force plot
shap.force_plot(explainer.expected_value, example_shap, example, 
                matplotlib=True, show=False)
plt.tight_layout()
plt.savefig('../models/shap_force_plot.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ SHAP plots sauvegardés")

## 11. Model Persistence

In [ ]:
print("\n" + "=" * 80)
print("SAUVEGARDE DU MODÈLE FINAL")
print("=" * 80)

# Sauvegarder le modèle
joblib.dump(best_xgb, '../models/best_model.pkl')
print("\n✅ Modèle sauvegardé: ../models/best_model.pkl")

# Sauvegarder les métadonnées
metadata = {
    'model_name': 'XGBoost (Optimized)',
    'model_type': 'XGBRegressor',
    'best_params': grid_search.best_params_,
    'metrics': results['XGBoost (Optimized)'],
    'feature_names': feature_names,
    'n_features': len(feature_names),
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'date': '2025-01-09'
}

with open('../models/model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✅ Métadonnées sauvegardées: ../models/model_metadata.json")

# Test de chargement
print("\n" + "=" * 80)
print("TEST DE CHARGEMENT DU MODÈLE")
print("=" * 80)

loaded_model = joblib.load('../models/best_model.pkl')
test_prediction = loaded_model.predict(X_test.head(1))

print(f"\n✅ Modèle chargé avec succès")
print(f"   Test prediction: {test_prediction[0]:,.2f} €")
print(f"   Actual value:    {y_test.iloc[0]:,.2f} €")
print(f"   Difference:      {abs(test_prediction[0] - y_test.iloc[0]):,.2f} € ({abs(test_prediction[0] - y_test.iloc[0])/y_test.iloc[0]*100:.2f}%)")

---
## Summary

### Modèles entraînés:
1. **Baseline** (Mean prediction)
2. **Linear Models**: LinearRegression, Ridge, Lasso, ElasticNet
3. **Tree-Based**: DecisionTree, RandomForest
4. **Boosting**: GradientBoosting, XGBoost, LightGBM, CatBoost

### Meilleur modèle:
- **XGBoost (Optimized)** après GridSearchCV
- Hyperparamètres optimisés par validation croisée (5-fold CV)
- Performance mesurée sur ensemble de test indépendant

### Interprétabilité:
- **Feature Importance**: Identification des variables les plus importantes
- **SHAP Values**: Explication détaillée de l'impact de chaque feature
- **Force Plots**: Explication de prédictions individuelles

### Production-ready:
- Modèle sauvegardé avec joblib (`.pkl`)
- Métadonnées complètes (JSON)
- Scaler sauvegardé pour preprocessing
- Reproductible (RANDOM_STATE=42)

### Prochaines étapes potentielles:
1. Déploiement avec FastAPI ou Flask
2. Monitoring en production
3. Ré-entraînement périodique
4. A/B testing de nouvelles features

---